In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models import LdaModel, LsiModel
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk

from pprint import pprint

import spacy

import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim

# Read data file

In [2]:
import os
cwd = os.getcwd()
cwd

'/Users/elgayaro/dev/teamA/scripts'

In [72]:
df = pd.read_csv ('../rawData/Sample Data -  Nov 25 2020 to January 07 2021.csv')
df

,Full Text
0,@hamcore_ First the boss metal zone schematics...
1,@sippygup its in the vaccine and 5g makes it m...
2,Him and all the others that do Not like and ha...
3,@TimRunsHisMouth Him and all the others that d...
4,@NYGovCuomo 99.97% survival rate no vaccine fo...
...,...
9953,Waiting for the train and I hear two mfs say “...
9954,I played my radical left avant garde music and...
9955,You are such a fag @MayorKeller And what the F...
9956,.@seanspicer hahahaha Trump's accomplishments?...


In [102]:
docs = np.array(df['Full Text'])
docs

array(['@hamcore_ First the boss metal zone schematics as 5g in a vaccine now hitler got stuff right??? Cmon man it’s only 10:42am',
       '@sippygup its in the vaccine and 5g makes it more powerful watch out',
       "Him and all the others that do Not like and hate DonaldTrump is just because They are part of the satanic Deep State illuminati and serves different??God, the same God that gave him the power to poisoning our food and water. That's Monsanto plus he is behind the Vaccines☠️ withVP",
       ...,
       'You are such a fag @MayorKeller And what the Fuck is that que que logo all about? What a stupid waste of time and money you are. No mask No distancing No testing No vaccine @GovMLG #scamdemic',
       ".@seanspicer hahahaha Trump's accomplishments?? The vaccine? That international companies have been working on since before Trump acknowledged there was a pandemic? #Hoax The economy? It rallied when Biden won and now when the transition started. Trump killed farmers.",
    

In [103]:
print(len(docs))
print(docs.shape)
print(range(len(docs)))

9958
(9958,)
range(0, 9958)


# Pre-processing

In [95]:
from nltk.tokenize import TweetTokenizer

nltk.download('stopwords')
# get a set of stopwords
stopwords = set(nltk.corpus.stopwords.words("english"))

from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/elgayaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/elgayaro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [104]:

def docs_preprocessor(docs):
    # Tokenize corpus
    tokenizer = TweetTokenizer(strip_handles=True)    
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Only keep tokens that contains alphanumeric characters, without symbols.
    docs = [[token for token in doc if token.isalnum()] for doc in docs]
    
    # Remove stopwords.
    docs = [[token for token in doc if token not in stopwords] for doc in docs]

    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 1] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [105]:
docs = docs_preprocessor(docs)
docs

[['first',
  'bos',
  'metal',
  'zone',
  'schematic',
  '5g',
  'vaccine',
  'hitler',
  'got',
  'stuff',
  'right',
  'cmon',
  'man'],
 ['vaccine', '5g', 'make', 'powerful', 'watch'],
 ['others',
  'like',
  'hate',
  'donaldtrump',
  'part',
  'satanic',
  'deep',
  'state',
  'illuminati',
  'serf',
  'different',
  'god',
  'god',
  'gave',
  'power',
  'poisoning',
  'food',
  'water',
  'monsanto',
  'plus',
  'behind',
  'vaccine',
  'withvp'],
 ['others',
  'like',
  'hate',
  'donaldtrump',
  'part',
  'satanic',
  'deep',
  'state',
  'illuminati',
  'serf',
  'different',
  'god',
  'god',
  'gave',
  'power',
  'poisoning',
  'food',
  'water',
  'monsanto',
  'plus',
  'behind',
  'vaccine',
  'withvp'],
 ['survival', 'rate', 'vaccine'],
 ['coincidence',
  'extremely',
  'rare',
  'say',
  'like',
  'autism',
  'million',
  'family',
  'dealing',
  'least',
  'one',
  'child',
  'autism',
  'sometimes',
  'mmr',
  'vaccine',
  'relation',
  'injury',
  'rare'],
 ['part

In [106]:
docs[0]

['first',
 'bos',
 'metal',
 'zone',
 'schematic',
 '5g',
 'vaccine',
 'hitler',
 'got',
 'stuff',
 'right',
 'cmon',
 'man']

In [107]:
docs[1]

['vaccine', '5g', 'make', 'powerful', 'watch']

In [108]:
docs[4]

['survival', 'rate', 'vaccine']

In [109]:
docs[5]

['coincidence',
 'extremely',
 'rare',
 'say',
 'like',
 'autism',
 'million',
 'family',
 'dealing',
 'least',
 'one',
 'child',
 'autism',
 'sometimes',
 'mmr',
 'vaccine',
 'relation',
 'injury',
 'rare']